## Accessing Chesapeake Land Use data with the Planetary Computer STAC API

The [Chesapeake Land Use](https://www.chesapeakeconservancy.org/conservation-innovation-center/high-resolution-data/land-use-data-project/) dataset is high-resolution 1-meter Land Use data product in raster format for the entire Chesapeake Bay watershed. The dataset was created by modifying the 2013-2014 high-resolution land cover dataset using 13 ancillary datasets including data on zoning, land use, parcel boundaries, landfills, floodplains, and wetlands. The product area encompasses over 250,000 square kilometers in New York, Pennsylvania, Maryland, Delaware, West Virginia, Virginia, and the District of Columbia. The dataset was created by the Chesapeake Conservancy Conservation Innovation Center for the Chesapeake Bay Program, which is a regional partnership of EPA, other federal, state, and local agencies and governments, nonprofits, and academic institutions that leads and directs Chesapeake Bay restoration efforts.

In this notebook, we'll demonstrate how to access and work with this data through the Planetary Computer. Documentation for this dataset is available at the [Planetary Computer Data Catalog](https://planetarycomputer.microsoft.com/dataset/chesapeake-lu).

### Environment setup

This notebook works with or without an API key, but you will be given more permissive access to the data with an API key.

* The [Planetary Computer Hub](https://planetarycomputer.microsoft.com/compute) is pre-configured to use your API key.
* To use your API key locally, set the environment variable `PC_SDK_SUBSCRIPTION_KEY` or use `planetary_computer.settings.set_subscription_key(<YOUR API Key>)`

In [1]:
from pystac_client import Client
from pystac.extensions.item_assets import ItemAssetsExtension

import planetary_computer as pc
import rich.table

### Query for data of interest

We'll query for 2013 Chesapeake Land Use data. 

In [2]:
catalog = Client.open("https://pct-apis-staging.westeurope.cloudapp.azure.com/stac/")

latitude = 39.29
longitude = -76.61
datetimes = [
    "2013"
]

buffer = 0.6
bbox = [longitude - buffer, latitude - buffer, longitude + buffer, latitude + buffer]
items = dict()

for datetime in datetimes:
    print(f"Fetching {datetime}")
    search = catalog.search(
        collections=["chesapeake-lu"],
        bbox=bbox,
        datetime=datetime
    )

items = list(search.get_items())
print(f"Returned {len(items)} Items")

Fetching 2013
Returned 166 Items


Each Item contains a single "data" asset with a URL to the location of the Asset data on [Azure Blob Storage](https://azure.microsoft.com/en-us/services/storage/blobs/). 

In [3]:
asset_href = items[0].assets["data"].href
print(asset_href)

https://landcoverdata.blob.core.windows.net/chesapeake/lu/BayWide_1m_LU_E1700000_N1960000.tif


### Available STAC Assets and Metadata

Let's check the available assets and metadata for the Chesapeake Land Use item. 

In [4]:
t = rich.table.Table("Key", "Title")
for k, asset in items[0].assets.items():
    t.add_row(k, asset.title)
t

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key              ┃ Title                           ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ data             │ Land Use Classes                │
│ tilejson         │ TileJSON with default rendering │
│ rendered_preview │ Rendered preview                │
└──────────────────┴─────────────────────────────────┘

In [5]:
table = rich.table.Table("key", "value")
for k, v in sorted(items[0].properties.items()):
    table.add_row(k, str(v))

table

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ key            ┃ value                                                                    ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ created        │ 2022-06-01T20:15:13.334677Z                                              │
│ datetime       │ None                                                                     │
│ end_datetime   │ 2014-12-31T23:59:59Z                                                     │
│ proj:epsg      │ None                                                                     │
│ proj:shape     │ [10000, 10000]                                                           │
│ proj:transform │ [1.0, 0.0, 1699999.1624823206, 0.0, -1.0, 1970000.8902923968]            │
│ proj:wkt2      │ PROJCS["USA_Contiguous_Albers_Equal_Area_Conic_USGS_version",GEOGCS["NA… │
│                │ 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],AUTHORITY["EPS… │
│ start_datetime │ 2013-01-01T00:00:00Z                                                     │
└────────────────┴──────────────────────────────────────────────────────────────────────────┘

### Displaying the data



In [6]:
# Classifications
collection = catalog.get_collection("chesapeake-lu")
ia = ItemAssetsExtension.ext(collection)
x = ia.item_assets["data"]

class_names = {
    x["description"]: x["value"] for x in x.properties["classification:classes"]
}
values_to_classes = {v: k for k, v in class_names.items()}
class_count = len(class_names)
class_names

{'Impervious Roads': 1,
 'Impervious Non-Roads': 2,
 'Tree Canopy over Impervious Surfaces': 3,
 'Water': 4,
 'Tidal Wetlands': 5,
 'Floodplain Wetlands': 6,
 'Other Wetlands': 7,
 'Forest': 8,
 'Tree Canopy over Turf Grass': 9,
 'Mixed Open': 10,
 'Fractional Turf (small)': 11,
 'Fractional Turf (medium)': 12,
 'Fractional Turf (large)': 13,
 'Fractional Impervious': 14,
 'Turf Grass': 15,
 'Cropland': 16,
 'Pasture/Hay': 17}

In [7]:
from IPython.display import Image

item = items[0]
Image(url=item.assets["rendered_preview"].href)